## Create flatten light curves for two different bands with two planets 
### This corresponds to the data used in Sect. 5.3 of pyaneti II paper
#### O. Barragán, July 2021

Load the libraries
(You have to have citlalicue installed in your computer `pip install citlalicue`)

In [ ]:
#Load libraries
from __future__ import print_function, division, absolute_import
import numpy as np
import matplotlib.pyplot as plt
#Import the light_curve class from citlalicue
#You can install citlalicue as pip install citlalicue
from citlalicue.citlalicue import citlali

#### start a random seed number

In [ ]:
seed = 28
np.random.seed(seed)

#### Let us create two citlali (light curves) instances, each one will be used for a different band


In [ ]:
#lc1 will be used for band 1, data will be created between 0 and 15 days
lc1 = citlali(tmin=0,tmax=15,cadence=5/60./24.)
#lc2 will be used for band 2, data will be created between 20 and 30 days
lc2 = citlali(tmin=20,tmax=30,cadence=5/60./24.) 

Let us plot the light curves to see how they look (flat with no signal at all)

In [ ]:
#as it is now, the lc1 and lc2 instances corresponds to the light curves of a uniform disc star
#let us plot it to see this
lc1.plot(fsx=20,fsy=5)
lc2.plot(fsx=20,fsy=5)

## Time to add the transit signals

We will create transits assuming we have two planets on circular orbits around a star with a density $\rho_* = 1.4\, \mathrm{g\,cm^{-3}}$

The first set of transit signals will be asigned to a planet with parameters

$$
T_0 = 4,
P  = 3,
b  = 0.25,
r_p = 0.025,
$$

where $T_0$ time of periastron, $P$ orbital period, $b$ impact parameter, $r_p$ planet radius scaled to the radius of the star.
The second set of transits will be caused by a planet with parameters

$$
T_0 = 3,
P  = 10,
b  = 0.7,
\rho_*  = 1400,
r_p/R_* = 0.05,
$$

And the first instrument will have quadractic limb darkening coefficients of 

$$
u_{1,b1} = 0.25,
u_{2,b1} = 0,
$$

and the second

$$
u_{1,b2} = 0.50,
u_{2,b2} = 0.25.
$$


Let us create the parameters with all the values

In [ ]:
#Stellar density
rho = 1.4

#Planet b parameters
T0b = 4.
Pb  = 3.
bb  = 0.25
rpb = 0.025

#Planet c parameters
T0c = 3.
Pc  = 10.
bc  = 0.7
rpc = 0.05

#Instrument b1 limb darkening coefficients
u1_b1 = 0.25
u2_b1 = 0

#Instrument b1 limb darkening coefficients
u1_b2 = 0.50
u2_b2 = 0.25


### Add both planets to lc1

We will add the planet signals to our lc1 light curve. We will assign both planets but only the limb darkening coeffcients corresponding to the b1 instrument.

In [ ]:
#We pass the parameters to cilatlicue as

#pars = [T0,P,b,rho,rp,u1,u2]

#Let us create a list with all the parameters for planet b
parsb = [T0b,Pb,bb,rho,rpb,u1_b1,u2_b1]

#Let us create a list with all the parameters for planet c
parsc = [T0c,Pc,bc,rho,rpc,u1_b1,u2_b1]

#Let us add the planet b by calling the add_transits method
lc1.add_transits(planet_parameters=parsb,planet_name='b')

#And let us add the planet b by calling the add_transits method
lc1.add_transits(planet_parameters=parsc,planet_name='c')

#re-plot the light curve to see the planets
lc1.plot(fsx=20,fsy=5)

We can see two different sets of planet signals in our lc1 plot.

### Add both planets to lc2

We will add the planet signals to our lc2 light curve. We will assign both planets but only the limb darkening coeffcients corresponding to the b2 instrument.

In [ ]:
#We pass the parameters to cilatlicue as

#pars = [T0,P,b,rho,rp,u1,u2]

#Let us create a list with all the parameters for planet b
parsb = [T0b,Pb,bb,rho,rpb,u1_b2,u2_b2]

#Let us create a list with all the parameters for planet c
parsc = [T0c,Pc,bc,rho,rpc,u1_b2,u2_b2]

#Let us add the planet b by calling the add_transits method
lc2.add_transits(planet_parameters=parsb,planet_name='b')

#And let us add the planet b by calling the add_transits method
lc2.add_transits(planet_parameters=parsc,planet_name='c')

#re-plot the light curve to see the planets
lc2.plot(fsx=20,fsy=5)

We can see two different sets of planet signals in our lc2 plot too.

## Time to add white noise

We will add white noise to both time-series. Our first light curve will have a typical error bar of 500 ppm, while our second light curve of 100 ppm. This can be done easily by calling the `add_white_noise` method for each citlali instance.

In [ ]:
#Define the error bar that we want to add to b1
error_bar_b1 = 500e-6

#Call the add_white_noise attribute
lc1.add_white_noise(std=error_bar_b1)

#Define the error bar that we want to add to b2
error_bar_b2 = 100e-6

#Call the add_white_noise attribute
lc2.add_white_noise(std=error_bar_b2)

#### Let us plot both light curves together to see the difference between both data sets

In [ ]:
plt.figure(1,figsize=(15,5))
plt.plot(lc1.time,lc1.flux,'o',alpha=0.5,label='band b1')
plt.plot(lc2.time,lc2.flux,'o',alpha=0.5,label='band b2')
plt.xlabel('time [d]')
plt.ylabel('Normalised flux')
plt.legend()
plt.show()

## Let us save the data in the multiband.dat file

Let us save the light curves in a file, taking care of assigning a different label for each band in order to model this data with pyaneti.

In [ ]:
fname = 'multiband.dat'
with open(fname,'w') as f:
    for i in range(len(lc1.time)):
        f.write("{} {} {} {} \n".format(lc1.time[i],lc1.flux[i],lc1.error_bar,'b1'))
    for i in range(len(lc2.time)):
        f.write("{} {} {} {} \n".format(lc2.time[i],lc2.flux[i],lc2.error_bar,'b2'))